In [2]:
import chardet
import pandas as pd

with open('./data/tax.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv('./data/tax.csv', encoding=result['encoding'])

df

,Date,Title,Numac,Link FR,Link NL,Text
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR DE\nELI - Navigatie systeem via een Euro...
2,1/16/2020,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2020040052,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR DE\nELI - Navigatie systeem via een Euro...
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
...,...,...,...,...,...,...
764,4/4/2022,SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021....,2022020551,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR DE\nELI - Navigatie systeem via een Euro...
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MA...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGI...,40002126,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL


In [3]:
df = df[df["Text"].str.contains("Duitse vertaling")==False]

In [66]:
df.shape

(662, 6)

In [67]:
df[df.isna().any(axis=1)]

,Date,Title,Numac,Link FR,Link NL,Text


In [68]:
df=df.drop(df.index[345])

In [4]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahboubeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahboubeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Dataset
from sklearn.datasets import fetch_20newsgroups
# Data manipulation
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 100
# Text preprocessing and modelling
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
# Visualisation
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(style='whitegrid', context='talk')
#from wordcloud import WordCloud
#import pyLDAvis
#import pyLDAvis.sklearn
# Warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
# Stopwords
stop_words = stopwords.words("dutch")
stop_words.extend(['bis', 'NL', 'FR','artikel', "januari", "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "oktober", "november", "december"])

In [71]:
print(f"Number of stop words: {len(stop_words)}")

Number of stop words: 114


In [6]:
def preprocess_text(document):
    """Preprocess document into normalised tokens."""
    # Tokenise words into alphabetic tokens with minimum length of 3
    tokeniser = RegexpTokenizer(r'[A-Za-z]{3,}')
    tokens = tokeniser.tokenize(document)
    
    # Tag words with POS tag
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags = pos_tag(tokens)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(t.lower(), pos=pos_map.get(p[0], 'v')) for t, p in pos_tags]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stop_words]
    return keywords

In [7]:
# Convert to document-term matrix
vectoriser = CountVectorizer(analyzer=preprocess_text)
example_matrix = vectoriser.fit_transform(df['Text'])
# Extract feature/term names
feature_names = vectoriser.get_feature_names()
# Inspect document-term matrix
pd.DataFrame.sparse.from_spmatrix(example_matrix, 
                                  columns=feature_names)

C:\Users\Mahboubeh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aafisc,aaj,aalst,aalter,aanbestedende,aanbestedingen,aanbevelen,aanbeveling,aanbevelingen,aanbevolen,...,zwalm,zwangere,zware,zwarte,zweden,zwembad,zwembaden,zwevegem,zwijndrecht,zwitserland
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
660,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(df['Text'])
# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)
X_train

,aafisc,aaj,aalst,aalter,aanbestedende,aanbestedingen,aanbevelen,aanbeveling,aanbevelingen,aanbevolen,...,zwiebeln,zwiesp,zwijndrecht,zwingend,zwingende,zwingenden,zwischen,zwischenmenschliche,zwischenzeit,zwitserland
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.051226,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.010882,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [83]:
# Build lda model
lda = LatentDirichletAllocation(n_components=768, random_state=0)
lda.fit(example_matrix)
# Inspect topics
def describe_topics(lda, feature_names, top_n_words=5, show_weight=False):
    """Print top n words for each topic from lda model."""
    normalised_weights = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    for i, weights in enumerate(normalised_weights):  
        print(f"********** Topic {i+1} **********")
        if show_weight:
            feature_weights = [*zip(np.round(weights, 4), feature_names)]
            feature_weights.sort(reverse=True)
            print(feature_weights[:top_n_words], '\n')
        else:
            top_words = [feature_names[i] for i in weights.argsort()[:-top_n_words-1:-1]]
            print(top_words, '\n')
describe_topics(lda, feature_names, show_weight=True)

********** Topic 1 **********
[(0.0375, 'staat'), (0.0308, 'erasmus'), (0.0277, 'ter'), (0.0187, 'euro'), (0.0185, 'zorgsector')] 

********** Topic 2 **********
[(0.059, 'zorgverleners'), (0.059, 'werkzaam'), (0.059, 'personeel'), (0.0295, 'derde'), (0.0295, 'corrigendum')] 

********** Topic 3 **********
[(0.0199, 'wib'), (0.0154, 'wet'), (0.0144, 'besluit'), (0.0139, 'bepaling'), (0.0133, 'eerste')] 

********** Topic 4 **********
[(0.0322, 'covid'), (0.0278, 'universitaire'), (0.025, 'ziekenhuizen'), (0.0217, 'meet'), (0.0214, 'besluit')] 

********** Topic 5 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 6 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 7 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

**

[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 71 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 72 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 73 **********
[(0.0242, 'covid'), (0.0221, 'maatregelen'), (0.0211, 'lid'), (0.0206, 'wet'), (0.016, 'advies')] 

********** Topic 74 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 75 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 76 **********
[(0.0383, 'ziekenhuizen'), (0.0263, 'voorschot'), (0.0223, 'besluit'), (0.0162, 'meet'), (0.0161, 'epidemie')] 

******

[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 129 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 130 **********
[(0.018, 'besluit'), (0.0147, 'meet'), (0.0093, 'koninklijk'), (0.0081, 'sectoren'), (0.0081, 'nemen')] 

********** Topic 131 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 132 **********
[(0.055, 'diamant'), (0.0325, 'wet'), (0.0264, 'synthetische'), (0.0183, 'economie'), (0.0179, 'handelaren')] 

********** Topic 133 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 134 **********
[(0.0264, 'meet'), (0.0209, 'vermeld'), (0.017, 'iii'), (0.0157, 'personeelsleden'), (0.0146, 'onderwijs')] 

****

[(0.029, 'covid'), (0.0258, 'nten'), (0.0256, 'pati'), (0.0165, 'meet'), (0.0149, 'overwegende')] 

********** Topic 189 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 190 **********
[(0.0812, 'gegevens'), (0.0653, 'hypothecaire'), (0.0588, 'betrekking'), (0.0541, 'inkomstenbelastingen'), (0.0456, 'meet')] 

********** Topic 191 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 192 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 193 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 194 **********
[(0.029, 'besluit'), (0.0185, 'kinderopvang'), (0.0182, 'meet'), (0.0174, 'buitenschoolse'), (0.0163, 'op

[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 250 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 251 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 252 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 253 **********
[(0.0455, 'ouderschapsbescherming'), (0.0308, 'verlof'), (0.0262, 'coronaverlof'), (0.0258, 'meet'), (0.0243, 'lid')] 

********** Topic 254 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 255 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'

...

[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 496 **********
[(0.0458, 'bevoegde'), (0.0284, 'autoriteiten'), (0.0214, 'arbiter'), (0.0166, 'autoriteit'), (0.0164, 'meet')] 

********** Topic 497 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 498 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 499 **********
[(0.0207, 'bedoeld'), (0.0145, 'lid'), (0.0143, 'art'), (0.0116, 'minister'), (0.0111, 'fagg')] 

********** Topic 500 **********
[(0.0289, 'steunregelingen'), (0.0289, 'journalismfund'), (0.0289, 'eenmalige'), (0.0289, 'delegatie'), (0.0289, 'definitieve')] 

********** Topic 501 **********
[(0.0308, 'toeristische'), (0.0274, 'toerisme'), (0.0215, 'activiteiten'), (0.0148, 'meet'), (0.0141, 'opera

[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 553 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 554 **********
[(0.1119, 'lening'), (0.0697, 'voordelen'), (0.0549, 'woord'), (0.0526, 'rentevoet'), (0.0474, 'aard')] 

********** Topic 555 **********
[(0.0891, 'wet'), (0.0682, 'koning'), (0.0667, 'strijd'), (0.0661, 'verspreiding'), (0.0657, 'verleent')] 

********** Topic 556 **********
[(0.0208, 'maatregelen'), (0.016, 'meet'), (0.0139, 'besluit'), (0.0133, 'minister'), (0.0132, 'wet')] 

********** Topic 557 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 558 **********
[(0.0386, 'wet'), (0.0336, 'artikelen'), (0.033, 'besluit'), (0.0281, 'lid'), (0.0281, 'eerste')] 

********** Topic 559 **********


[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 610 **********
[(0.0939, 'brussel'), (0.0885, 'fiscaliteit'), (0.0479, 'directeur'), (0.0378, 'aanwezig'), (0.0305, 'overheidsdienst')] 

********** Topic 611 **********
[(0.0308, 'wet'), (0.0278, 'pensioenen'), (0.0208, 'houdende'), (0.0186, 'werkgever'), (0.0177, 'covid')] 

********** Topic 612 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 613 **********
[(0.0283, 'noodmaatregelen'), (0.0283, 'nietig'), (0.0283, 'inzake'), (0.0283, 'bovenvermeld'), (0.0283, 'beschouwd')] 

********** Topic 614 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 615 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.

[(0.0273, 'ontwerp'), (0.0208, 'lid'), (0.0193, 'geneesmiddelen'), (0.0164, 'richtlijn'), (0.0141, 'nemen')] 

********** Topic 667 **********
[(0.0437, 'online'), (0.0388, 'leerlingen'), (0.0365, 'uitrusting'), (0.0292, 'riziv'), (0.0292, 'dotatie')] 

********** Topic 668 **********
[(0.0344, 'maatregelen'), (0.028, 'bevoegde'), (0.0276, 'covid'), (0.0264, 'pandemie'), (0.0197, 'dienstbetrekking')] 

********** Topic 669 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 670 **********
[(0.0276, 'per'), (0.025, 'kostprijs'), (0.0191, 'federale'), (0.0166, 'gevaccineerde'), (0.0166, 'euro')] 

********** Topic 671 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 672 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 



[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 725 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 726 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 727 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 728 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 729 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden'), (0.0001, 'zwembad')] 

********** Topic 730 **********
[(0.0001, 'zwitserland'), (0.0001, 'zwijndrecht'), (0.0001, 'zwevegem'), (0.0001, 'zwembaden')

In [8]:
# Build lda model
lda = LatentDirichletAllocation(n_components=2, random_state=0)
lda.fit(example_matrix)
# Inspect topics
def describe_topics(lda, feature_names, top_n_words=5, show_weight=False):
    """Print top n words for each topic from lda model."""
    normalised_weights = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    for i, weights in enumerate(normalised_weights):  
        print(f"********** Topic {i+1} **********")
        if show_weight:
            feature_weights = [*zip(np.round(weights, 4), feature_names)]
            feature_weights.sort(reverse=True)
            print(feature_weights[:top_n_words], '\n')
        else:
            top_words = [feature_names[i] for i in weights.argsort()[:-top_n_words-1:-1]]
            print(top_words, '\n')
describe_topics(lda, feature_names, show_weight=True)


********** Topic 1 **********
[(0.0153, 'meet'), (0.0149, 'besluit'), (0.0126, 'covid'), (0.0072, 'gelet'), (0.0069, 'maatregelen')] 

********** Topic 2 **********
[(0.0161, 'besluit'), (0.0155, 'meet'), (0.0135, 'lid'), (0.0116, 'eerste'), (0.0111, 'art')] 



In [9]:
df[['topic1', 'topic2']] = lda.transform(example_matrix)
df

<ipython-input-9-b9e02361b369>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['topic1', 'topic2']] = lda.transform(example_matrix)
C:\Users\Mahboubeh\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,Date,Title,Numac,Link FR,Link NL,Text,topic1,topic2
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-14&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-14&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.161222,0.838778
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEMBRE 2019. - Arrêté ministériel approuvant le dossi...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-24&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-24&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-24\nNumac : 2020010214\nMINISTER...,0.003796,0.996204
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté royal remédiant au double emploi de l...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.003987,0.996013
5,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté ministériel adaptant l'arrêté ministé...,2020020094,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.005366,0.994634
6,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministration générale de la Fiscalité. - Impôts sur les reven...,2020010193,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-28\nNumac : 2020010193\nFEDERALE...,0.000305,0.999695
...,...,...,...,...,...,...,...,...
763,4/4/2022,SERVICE PUBLIC FEDERAL FINANCES\n23 FEVRIER 2022. - Loi visant à mettre en oeuvre le règlement(U...,2022040186,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State Kamer van volksvertegenwoordige...,0.000433,0.999567
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONCERTATION SOCIALE\n31 MARS 2022. - Arrêté royal pro...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.543587,0.456413
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MARS 2022. - Arrêté royal modifiant l'arrêté royal n...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.938130,0.061870
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGION DE BRUXELLES-CAPITALE\nSERVICE PUBLIC REGIONAL ...,40002126,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL,0.500000,0.500000


In [11]:
df['top'] = df.iloc[:, 6:8].idxmax(axis=1)
df['prop'] = df.iloc[:, 6:8].max(axis=1)
df

<ipython-input-11-b42af253767d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['top'] = df.iloc[:, 6:8].idxmax(axis=1)
<ipython-input-11-b42af253767d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prop'] = df.iloc[:, 6:8].max(axis=1)


,Date,Title,Numac,Link FR,Link NL,Text,topic1,topic2,top,prop
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-14&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-14&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.161222,0.838778,topic2,0.838778
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEMBRE 2019. - Arrêté ministériel approuvant le dossi...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-24&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-24&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-24\nNumac : 2020010214\nMINISTER...,0.003796,0.996204,topic2,0.996204
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté royal remédiant au double emploi de l...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.003987,0.996013,topic2,0.996013
5,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté ministériel adaptant l'arrêté ministé...,2020020094,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.005366,0.994634,topic2,0.994634
6,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministration générale de la Fiscalité. - Impôts sur les reven...,2020010193,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-28\nNumac : 2020010193\nFEDERALE...,0.000305,0.999695,topic2,0.999695
...,...,...,...,...,...,...,...,...,...,...
763,4/4/2022,SERVICE PUBLIC FEDERAL FINANCES\n23 FEVRIER 2022. - Loi visant à mettre en oeuvre le règlement(U...,2022040186,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State Kamer van volksvertegenwoordige...,0.000433,0.999567,topic2,0.999567
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONCERTATION SOCIALE\n31 MARS 2022. - Arrêté royal pro...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.543587,0.456413,topic1,0.543587
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MARS 2022. - Arrêté royal modifiant l'arrêté royal n...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.938130,0.061870,topic1,0.938130
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGION DE BRUXELLES-CAPITALE\nSERVICE PUBLIC REGIONAL ...,40002126,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov

In [12]:
topic_mapping = {'topic1': 'tax', 'topic2': 'not tax'}
df['topic'] = df['top'].map(topic_mapping)
df

<ipython-input-12-986a77075639>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic'] = df['top'].map(topic_mapping)


,Date,Title,Numac,Link FR,Link NL,Text,topic1,topic2,top,prop,topic
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXELLES-CAPITALE\n8 JANVIER 2020. - Arrêté ministériel...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-14&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-14&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.161222,0.838778,topic2,0.838778,not tax
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEMBRE 2019. - Arrêté ministériel approuvant le dossi...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-24&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-24&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-24\nNumac : 2020010214\nMINISTER...,0.003796,0.996204,topic2,0.996204,not tax
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté royal remédiant au double emploi de l...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.003987,0.996013,topic2,0.996013,not tax
5,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 2020. - Arrêté ministériel adaptant l'arrêté ministé...,2020020094,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.005366,0.994634,topic2,0.994634,not tax
6,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministration générale de la Fiscalité. - Impôts sur les reven...,2020010193,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2020-01-28&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2020-01-28&caller=list...,NL FR\n\neinde eerste woord laatste woord\nPublicatie : 2020-01-28\nNumac : 2020010193\nFEDERALE...,0.000305,0.999695,topic2,0.999695,not tax
...,...,...,...,...,...,...,...,...,...,...,...
763,4/4/2022,SERVICE PUBLIC FEDERAL FINANCES\n23 FEVRIER 2022. - Loi visant à mettre en oeuvre le règlement(U...,2022040186,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State Kamer van volksvertegenwoordige...,0.000433,0.999567,topic2,0.999567,not tax
765,4/4/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONCERTATION SOCIALE\n31 MARS 2022. - Arrêté royal pro...",2022201913,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.543587,0.456413,topic1,0.543587,tax
766,4/4/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MARS 2022. - Arrêté royal modifiant l'arrêté royal n...,2022031428,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub_date=2022-04-04&caller=list&numa...,http://www.ejustice.just.fgov.be/cgi/article_body.pl?language=nl&pub_date=2022-04-04&caller=list...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgeving\nRaad van State\nELI - Navigatie systeem via ee...,0.938130,0.061870,topic1,0.938130,tax
767,4/4/2022,GOUVERNEMENTS DE COMMUNAUTE ET DE REGION\nREGION DE BRUXELLES-CAPITALE\nSERVICE PUBLIC REGIONAL ...,40002126,http://www.ejustice.just.fgov.be/cgi/article.pl?language=fr&pub